In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import os

In [ ]:
data_dir="/content/drive/MyDrive/Colab Notebooks/Tensorflow_CNN/Simpson/Simpson/Simp_dataset"

train_dir=os.path.join(data_dir, "train")
test_dir=os.path.join(data_dir, "test")

In [ ]:
os.listdir(train_dir)

['maggie_simpson',
 'ned_flanders',
 'marge_simpson',
 'milhouse_van_houten',
 'homer_simpson',
 'lisa_simpson',
 'edna_krabappel',
 'chief_wiggum',
 'abraham_grampa_simpson',
 'bart_simpson']

In [ ]:
#Labelisation des données avec ImageDataGenerator


import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator


train_data_generator=ImageDataGenerator(rescale=1/255.0,)
test_data_generator=ImageDataGenerator(rescale=1/255.0,)

#Générer le training
train_generator=train_data_generator.flow_from_directory(
    directory=train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",
)


#Générer le test
test_generator=test_data_generator.flow_from_directory(
    directory=test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode="categorical",

)

Found 11221 images belonging to 10 classes.
Found 485 images belonging to 10 classes.


In [ ]:
train_generator.class_indices

{'abraham_grampa_simpson': 0,
 'bart_simpson': 1,
 'chief_wiggum': 2,
 'edna_krabappel': 3,
 'homer_simpson': 4,
 'lisa_simpson': 5,
 'maggie_simpson': 6,
 'marge_simpson': 7,
 'milhouse_van_houten': 8,
 'ned_flanders': 9}

In [ ]:
from keras.utils import to_categorical


#Appliquer le one_hot encoding
#train_images,train_labels=next(train_generator)


train_generator.reset()
train_images,train_labels=next(train_generator)

train_labels = to_categorical(train_labels, num_classes=10)

print(train_labels)



[[[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]]

 ...

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [0. 1. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  ...
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]]


In [ ]:
#Entrainement du modèle

from tensorflow.keras.models import Sequential

model=Sequential([
    #Extraction des caractéristiques

    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), padding="same", activation="relu", input_shape=(224,224,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation="relu"),
    tf.keras.layers.MaxPooling2D(2,2),


    #Applatir
    tf.keras.layers.Flatten(),

    #Dense
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=10, activation="softmax"),


])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 55, 55, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 53, 53, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss="categorical_crossentropy",
               metrics=['accuracy'])

In [ ]:
#definition des callbacks: ModelCheckpoint :  Permet d'enregistrer le meilleur modème à chaque itération

from tensorflow.keras.callbacks import ModelCheckpoint

best_model="best_model.h5"

model_ckp=ModelCheckpoint(filepath=best_model,
                          monitor="val_accuracy",
                          mode="max",
                          save_best_only=True)

In [ ]:
#definition des callbacks: Early Stopping (Permet d'arrêter l'entrainement si on obtient plus de meilleur modèle après 5 itérations)
#Vous pouvez modifier la valeur 5 au nivau de "patience"

stop=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=5)

In [ ]:
#h=model.fit(train_generator,epochs=10, validation_data=test_generator, callbacks=[model_ckp, stop])

In [ ]:
#Sauvegarder le modèle
  model.save("/content/drive/MyDrive/Colab Notebooks/Tensorflow_CNN/Simpson/Simpson/best_model.h5")